<a href="https://colab.research.google.com/github/siranjeevi21/Car-Detector/blob/main/Covid_prediction_using_cnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget http://cb.lk/covid_19

--2021-05-22 05:41:18--  http://cb.lk/covid_19
Resolving cb.lk (cb.lk)... 159.89.170.74
Connecting to cb.lk (cb.lk)|159.89.170.74|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://cb.lk/covid_19 [following]
--2021-05-22 05:41:18--  https://cb.lk/covid_19
Connecting to cb.lk (cb.lk)|159.89.170.74|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://www.dropbox.com/s/7rjw6oet4za01op/CovidDataset-20200427T133042Z-001.zip?dl=0 [following]
--2021-05-22 05:41:19--  https://www.dropbox.com/s/7rjw6oet4za01op/CovidDataset-20200427T133042Z-001.zip?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.6.18, 2620:100:601c:18::a27d:612
Connecting to www.dropbox.com (www.dropbox.com)|162.125.6.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/7rjw6oet4za01op/CovidDataset-20200427T133042Z-001.zip [following]
--2021-05-22 05:41:19--  https://www.dropbox.com/s/raw/7

In [2]:
!unzip covid_19

Archive:  covid_19
  inflating: CovidDataset/Val/Covid/88de9d8c39e946abd495b37cd07d89e5-6531-0.jpg  
  inflating: CovidDataset/Val/Covid/covid-19-pneumonia-8.jpg  
  inflating: CovidDataset/Val/Covid/85E52EB3-56E9-4D67-82DA-DEA247C82886.jpeg  
  inflating: CovidDataset/Val/Covid/4-x-day1.jpg  
  inflating: CovidDataset/Val/Covid/covid-19-pneumonia-53.jpg  
  inflating: CovidDataset/Val/Covid/4e43e48d52c9e2d4c6c1fb9bc1544f_jumbo.jpeg  
  inflating: CovidDataset/Val/Covid/covid-19-pneumonia-35-1.jpg  
  inflating: CovidDataset/Val/Covid/53EC07C9-5CC6-4BE4-9B6F-D7B0D72AAA7E.jpeg  
  inflating: CovidDataset/Val/Covid/6CB4EFC6-68FA-4CD5-940C-BEFA8DAFE9A7.jpeg  
  inflating: CovidDataset/Val/Covid/58cb9263f16e94305c730685358e4e_jumbo.jpeg  
  inflating: CovidDataset/Val/Covid/6b3bdbc31f65230b8cdcc3cef5f8ba8a-40ac-0.jpg  
  inflating: CovidDataset/Val/Covid/auntminnie-d-2020_01_28_23_51_6665_2020_01_28_Vietnam_coronavirus.jpeg  
  inflating: CovidDataset/Val/Covid/6b44464d-73a7-4cf3-bbb6-ffe7

In [3]:
import numpy as np
import matplotlib.pyplot as plt
import keras
from keras.layers import *
from keras.models import *
from keras.preprocessing import image

In [4]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(224, 224, 3)))
model.add(Conv2D(64, (3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))


model.compile(loss=keras.losses.binary_crossentropy, optimizer='adam', metrics=['accuracy'])

In [5]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 222, 222, 32)      896       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 220, 220, 64)      18496     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 110, 110, 64)      0         
_________________________________________________________________
dropout (Dropout)            (None, 110, 110, 64)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 108, 108, 128)     73856     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 54, 54, 128)       0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 54, 54, 128)       0

In [11]:
train_datagen = image.ImageDataGenerator(
    rescale = 1./255,
    shear_range = 0.2, 
    zoom_range = 0.2,
    horizontal_flip = True,
)

test_dataset = image.ImageDataGenerator(rescale=1./255)

In [12]:
train_generator = train_datagen.flow_from_directory(
    'CovidDataset/Train',
    target_size = (224, 224),
    batch_size = 32,
    class_mode = 'binary'
)

Found 224 images belonging to 2 classes.


In [13]:
train_generator.class_indices

{'Covid': 0, 'Normal': 1}

In [14]:
validation_generator = test_dataset.flow_from_directory(
    'CovidDataset/Val',
    target_size = (224, 224),
    batch_size = 32,
    class_mode = 'binary'
)

Found 60 images belonging to 2 classes.


In [19]:
hist = model.fit_generator(
    train_generator,
    steps_per_epoch=5,
    epochs = 100,
    validation_data = validation_generator,
    validation_steps=2
)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/100
5/5 [==============================] - 8s 2s/step - loss: 0.0669 - accuracy: 0.9688 - val_loss: 0.0618 - val_accuracy: 0.9667
Epoch 2/100
5/5 [==============================] - 7s 2s/step - loss: 0.0801 - accuracy: 0.9875 - val_loss: 0.0655 - val_accuracy: 0.9500
Epoch 3/100
5/5 [==============================] - 7s 1s/step - loss: 0.0530 - accuracy: 0.9937 - val_loss: 0.0618 - val_accuracy: 0.9667
Epoch 4/100
5/5 [==============================] - 7s 2s/step - loss: 0.0698 - accuracy: 0.9750 - val_loss: 0.0998 - val_accuracy: 0.9667
Epoch 5/100
5/5 [==============================] - 7s 2s/step - loss: 0.0627 - accuracy: 0.9688 - val_loss: 0.0937 - val_accuracy: 0.9667
Epoch 6/100
5/5 [==============================] - 7s 2s/step - loss: 0.0394 - accuracy: 0.9875 - val_loss: 0.1341 - val_accuracy: 0.9667
Epoch 7/100
5/5 [==============================] - 7s 2s/step - loss: 0.0911 - accuracy: 0.9625 - val_loss: 0.0898 - val_accuracy: 0.9500
Epoch 8/100
5/5 [=================

In [20]:
import tensorflow as tf

In [22]:
tf.keras.models.save_model(model, 'my_model2.hdfs')

INFO:tensorflow:Assets written to: my_model2.hdfs/assets


In [23]:
!pip install streamlit

     |████████████████████████████████| 8.2MB 13.7MB/s 
     |████████████████████████████████| 174kB 52.0MB/s 
     |████████████████████████████████| 112kB 56.6MB/s 
     |████████████████████████████████| 81kB 11.2MB/s 
     |████████████████████████████████| 92kB 13.4MB/s 
     |████████████████████████████████| 4.2MB 42.9MB/s 
     |████████████████████████████████| 71kB 11.3MB/s 
     |████████████████████████████████| 122kB 58.8MB/s 
  Created wheel for blinker: filename=blinker-1.4-cp37-none-any.whl size=13448 sha256=0646b51466b39b14988d87b3411f9ae42361d344a2fa748676cfef9a73616ce7
  Stored in directory: /root/.cache/pip/wheels/92/a0/00/8690a57883956a301d91cf4ec999cc0b258b01e3f548f86e89
Successfully built blinker
ERROR: google-colab 1.0.0 has requirement ipykernel~=4.10, but you'll have ipykernel 5.5.5 which is incompatible.
  Found existing installation: click 8.0.0
    Uninstalling click-8.0.0:
      Successfully uninstalled click-8.0.0
  Found existing installation: ipykernel

In [1]:
%%writefile app.py

import streamlit as st
st.set_option('depreciation.showfileUploaderEncoding', False)
@st.cache(allow_output_mutation=True)
def load_model():
  model=tf.keras.models.load_model('/content/my_model2.hdfs')
  return model
model=load_model()
st.write("""
      # Covid-19 Detector

""")  

file = st.file_uploader("Upload your Chest X-ray file to detect Covid-19", type=["jpg", "png"])
import cv2
import numpy as np
from PIL import Image, ImageOps
def import_and_predict(image_data, model):
  size=(224, 224)
  image = ImageOps.fit(image_data, size, Image.ANTIALIAS)
  img = np.array(image)
  img_reshape = img[np.newaxis,...]
  prediction = model.predict(img_reshape)

  return prediction
if file is None:
  st.text(image, use_coumn_width=True)
  predictions = import_and_predict(image, model)
  class_names=["Covid", "Normal"]
  string="The result was most likely to be: "+class_names[np.argmax(predictions)]
  st.success(string)
  


Writing app.py


In [4]:
!ngrok authtoken 1sseG6auv7uZLE3URPDiN9ID1rF_5aZTfsVUq6W8bmGUoogAa

/bin/bash: ngrok: command not found
